<a href="https://colab.research.google.com/github/ncminhbka/ML-Projects/blob/main/LSTM_text_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Nhớ rằng LSTM hoạt động không phụ thuộc seqlen, có bao nhiêu nó tạo bấy nhiêu cell và áp dụng cùng bộ trọng số. Khi train do batch cố định kích thước nên ta phải chọn seqlen cố định, nhưng khi infer input của ta dài bao nhiêu cũng được. Nếu bạn thích thì bỏ batch đi, train từng sample một thì seqlen sẽ được tùy ý (haha)

In [ ]:
!wget -O shakespeare.txt https://www.gutenberg.org/cache/epub/1513/pg1513.txt


--2025-10-12 15:03:43--  https://www.gutenberg.org/cache/epub/1513/pg1513.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 169546 (166K) [text/plain]
Saving to: ‘shakespeare.txt’

shakespeare.txt     100%[===================>] 165.57K   624KB/s    in 0.3s    

2025-10-12 15:03:44 (624 KB/s) - ‘shakespeare.txt’ saved [169546/169546]



trước khi chạy cần vào shakespeare.txt xóa các dòng không liên quan (ở đầu và cuối)

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [ ]:
filename = 'shakespeare.txt'
with open(filename, 'r', encoding = 'utf-8') as f:
    raw_text = f.read()
raw_text = raw_text.lower()

import re

# Loại bỏ các dòng toàn chữ in hoa (thường là tiêu đề, nhân vật, tên cảnh)
clean_text = []
for line in raw_text.splitlines():
    line = line.strip()
    # Bỏ dòng rỗng hoặc dòng toàn chữ in hoa (SCENE I, ROMEO, CAPULET, v.v.)
    if not line or line.isupper():
        continue
    clean_text.append(line)

# Gộp lại thành một chuỗi duy nhất
raw_text = " ".join(clean_text)



print(raw_text[:1000])



#charecter-level
vocab = sorted(list(set(raw_text)))
char_to_idx = {ch:i for i,ch in enumerate(vocab)}
idx_to_char = {i:ch for i,ch in enumerate(vocab)}

the tragedy of romeo and juliet by william shakespeare contents the prologue. act i scene i. a public place. scene ii. a street. scene iii. room in capulet’s house. scene iv. a street. scene v. a hall in capulet’s house. act ii chorus. scene i. an open place adjoining capulet’s garden. scene ii. capulet’s garden. scene iii. friar lawrence’s cell. scene iv. a street. scene v. capulet’s garden. scene vi. friar lawrence’s cell. act iii scene i. a public place. scene ii. a room in capulet’s house. scene iii. friar lawrence’s cell. scene iv. a room in capulet’s house. scene v. an open gallery to juliet’s chamber, overlooking the garden. act iv scene i. friar lawrence’s cell. scene ii. hall in capulet’s house. scene iii. juliet’s chamber. scene iv. hall in capulet’s house. scene v. juliet’s chamber; juliet on the bed. act v scene i. mantua. a street. scene ii. friar lawrence’s cell. scene iii. a churchyard; in it a monument belonging to the capulets. dramatis personæ escalus, prince of veron

In [ ]:
len(vocab)

44

In [ ]:
len(raw_text)

141092

In [ ]:
window_size = 100
dataX = []
dataY = []
for i in range(0, len(raw_text)-window_size):
    X = raw_text[i:i+window_size]
    Y = raw_text[i+window_size]
    dataX.append([char_to_idx[ch] for ch in X])
    dataY.append(char_to_idx[Y])

In [ ]:
import numpy as np
dataX = np.array(dataX)
dataY = np.array(dataY)

In [ ]:
dataX.shape #n_sample, sqlen

(140992, 100)

In [ ]:
dataY.shape

(140992,)

In [ ]:
vocab_size = len(vocab)

In [ ]:
#dataset
class TextDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype = torch.long) #embedding
        self.y = torch.tensor(y, dtype = torch.long) #cross entropy
    def __len__(self):
        return len(self.y)
    def __getitem__(self, idx):
        sample = self.X[idx], self.y[idx]
        return sample

In [ ]:
train_dataset = TextDataset(dataX, dataY)

In [ ]:
batch_size = 128
dataloader = DataLoader(train_dataset, batch_size, shuffle = True)

In [ ]:
for X_batch, y_batch in dataloader:
    print(X_batch.shape)
    print(y_batch.shape)
    break

torch.Size([128, 100])
torch.Size([128])


In [ ]:
class TextGenerator(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, vocab_size, dropout_prob):
        super(TextGenerator, self).__init__()
        self.embedding = nn.Embedding(vocab_size, input_size)
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first = True)
        self.norm = nn.LayerNorm(hidden_size)
        self.dropout = nn.Dropout(dropout_prob)
        self.fc = nn.Linear(hidden_size, vocab_size)
    def forward(self, x):
        x = self.embedding(x)
        x, hn = self.lstm(x)
        x = x[:, -1, :] # last cell of the last layer
        x = self.norm(x)
        x = self.dropout(x)
        x = self.fc(x)
        return x

In [ ]:
input_size = 64
hidden_size = 256
num_layers = 3
dropout_prob = 0.2
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = TextGenerator(input_size, hidden_size, num_layers, vocab_size, dropout_prob).to(device)

In [ ]:
dummy_tensor = torch.randint(0, vocab_size, (1, 100)).to(device)
output = model(dummy_tensor)
print(output.argmax(dim = -1).item())

15


In [ ]:
epochs = 50
lr = 1e-3
optimizer = torch.optim.Adam(model.parameters(), lr = lr)
criterion = nn.CrossEntropyLoss()
'''
🔹 nn.CrossEntropyLoss yêu cầu y_batch phải là kiểu torch.long
🔹 và y_pred phải là torch.float
Nó tự thực hiện softmax + log + chọn đúng index — không cần one-hot.
CrossEntropyLoss cần (batch,).
'''

'\n🔹 nn.CrossEntropyLoss yêu cầu y_batch phải là kiểu torch.long\n🔹 và y_pred phải là torch.float\nNó tự thực hiện softmax + log + chọn đúng index — không cần one-hot.\nCrossEntropyLoss cần (batch,).\n'

In [ ]:
for epoch in range(epochs):
    losses = 0
    for X_batch, y_batch in dataloader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        y_pred = model(X_batch)
        loss = criterion(y_pred, y_batch)
        losses += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    losses /= len(dataloader)
    print(f'Epoch {epoch+1}/{epochs}, Loss: {losses}')

Epoch 1/50, Loss: 1.9618118106778002
Epoch 2/50, Loss: 1.5595216322724053
Epoch 3/50, Loss: 1.4222104805783655
Epoch 4/50, Loss: 1.324494192799293
Epoch 5/50, Loss: 1.2417488836441628
Epoch 6/50, Loss: 1.1614220718939379
Epoch 7/50, Loss: 1.0849296617205044
Epoch 8/50, Loss: 1.0080804107626207
Epoch 9/50, Loss: 0.9293790420148853
Epoch 10/50, Loss: 0.8544306745871009
Epoch 11/50, Loss: 0.7821856476975007
Epoch 12/50, Loss: 0.7132757025383778
Epoch 13/50, Loss: 0.6480866976744899
Epoch 14/50, Loss: 0.5911755326547121
Epoch 15/50, Loss: 0.5425129911427922
Epoch 16/50, Loss: 0.49816911495856925
Epoch 17/50, Loss: 0.46224192025548966
Epoch 18/50, Loss: 0.431004138964274
Epoch 19/50, Loss: 0.39952911628569104
Epoch 20/50, Loss: 0.37896235903789255
Epoch 21/50, Loss: 0.3562955996816041
Epoch 22/50, Loss: 0.3426946895957642
Epoch 23/50, Loss: 0.32529385193181776
Epoch 24/50, Loss: 0.31843800470232964
Epoch 25/50, Loss: 0.2980880004281141
Epoch 26/50, Loss: 0.29426870101592933
Epoch 27/50, Los

In [ ]:
seq_length = 100
start = np.random.randint(0, len(raw_text)-seq_length)
prompt = raw_text[start:start+seq_length]
prompt = 'young men’s love then lies not truly in their hearts, but in their eyes'

In [ ]:
indexed_prompt = [char_to_idx[ch] for ch in prompt]


In [ ]:

model.eval()
print(f'Prompt: {prompt} \n')
with torch.no_grad():
    for i in range(1000):
        X = indexed_prompt
        X = np.array(X)
        X = np.expand_dims(X, axis = 0)
        X = torch.tensor(X, dtype = torch.long).to(device)
        pred = model(X)
        index = pred.argmax(dim = -1).item()
        next_char = idx_to_char[index]
        print(next_char, end='')

        indexed_prompt.append(index)
        indexed_prompt = indexed_prompt[1:]
print()
print("DONE")

Prompt: young men’s love then lies not truly in their hearts, but in their eyes 

, and but kill one life. i beg for juliet of an ill cook that cannot lick his fingers goes not with me. citiz’d than she will stay with thee, and not thy will and her beauty sighs from heaven clears, thy old groans you are well. [_exit._] scene ii. hall in capulet’s house. act ii chorus. scene i. an open place adjoining capulet’s garden. enter romeo. romeo. father, what news? what is the prince’s doom? friar lawrence. i am the greatest, able to bed, acting of the watch with lovers’ brains, and then they do speak to tybalt, juliet pin’d to her heaven she said tybalt’s death, and then i have voulnt, and where me will i remain with worms that are thy chambermaids. o, here will i set up my everlasting rest; and shake the yoke of inauspicious stars that cannot lick his fingers goes not with me. citiz’d than she will stay with thee, and not thy will and her beauty sighs from heaven clears, thy old groans you ar